In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import glob

def predict_batch(self, item, rm_type_tfms=None, with_input=False):
    dl = self.dls.test_dl(item, rm_type_tfms=rm_type_tfms, num_workers=os.cpu_count())
    ret = self.get_preds(dl=dl)
    return ret

Learner.predict_batch = predict_batch

In [ ]:
!git clone https://github.com/d-li14/efficientnetv2.pytorch.git effnet

In [ ]:
df = pd.read_csv("../input/g2net-gravitational-wave-detection/training_labels.csv")
df["id"] = df["id"] + ".png"

skf = StratifiedKFold(n_splits=5)
t = df.target

for k, (train_index, test_index) in enumerate(skf.split(np.zeros(len(t)), t)):
    train = df.loc[train_index]
    test = df.loc[test_index]
    
    df_new = pd.DataFrame()
    
    train.reset_index(drop=True, inplace=True)
    df_new = pd.concat([train, pd.DataFrame(np.array([False] * len(train)))], axis=1)
    df_new.columns = ["image", "labels", "is_valid"]
    
    test.reset_index(drop=True, inplace=True)
    test = pd.concat([test, pd.DataFrame(np.array([True] * len(test)))], axis=1)
    test.columns = ["image", "labels", "is_valid"]
    
    df_new = pd.concat([df_new, test], axis=0)
    df_new.sort_values(by="image", inplace=True)
    df_new.reset_index(drop=True, inplace=True)
    
    df_new.to_csv(f"train_fold_{k}.csv", header=True, index=False)

In [ ]:
df_new.head()

In [ ]:
from PIL import Image
Image.open("../input/g2net-melspec-image/g2net_melspec/00000e74ad.png")

In [ ]:
source = Path("../input/g2net-melspec-image/")

datablock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                     splitter=ColSplitter(),
                     get_x=ColReader(0, pref=source/"g2net_melspec"),
                     get_y=ColReader(1),
                     item_tfms=Resize((56, 128)),
                     batch_tfms=[
                         IntToFloatTensor,
                         Normalize.from_stats(*imagenet_stats)
                     ])

df_final = pd.read_csv("train_fold_0.csv")
dls = datablock.dataloaders(df_final, bs=64, num_workers=os.cpu_count())
dls.show_batch()

In [ ]:
import torch 
import torch.nn as nn
from effnet.effnetv2 import effnetv2_m

cnn_model = effnetv2_m(num_classes=2)

try:
    del learn
    torch.cuda.empty_cache()
except Exception:
    pass

# Skip using FP16 since P100 does not have Tensor Cores. 
learn = Learner(dls, cnn_model, metrics=[accuracy, F1Score(), RocAucBinary()])
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, lr_max=3e-5)

In [ ]:
learn.fit_one_cycle(2, slice(1e-5, 3e-5))

In [ ]:
learn.fit_one_cycle(2, slice(1e-6, 6e-6))

In [ ]:
learn.export("/kaggle/working/effnetv2_m_melspec.pkl")